In [ ]:
%%writefile requirments.txt
datasets
trl
peft

In [ ]:
!pip install -r requirments.txt

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
from peft import LoraConfig
from transformers import AutoModelForCausalLM
import torch
import torch.nn as nn
from trl import SFTTrainer
from transformers import TrainingArguments
from peft import PeftModel
from transformers import AutoModelForCausalLM

In [ ]:
ds = load_dataset("FinGPT/fingpt-sentiment-train", split="train")
print(ds[0])

In [ ]:
def sample_text(example):
    text = (
        f"{example['instruction']}\n"
        f"Content: {example['input']}\n"
        f"Sentiment: {example['output']}"
    )
    return {"text": text}


In [ ]:
ds = ds.map(sample_text, remove_columns=["instruction", "input", "output"])
print(next(iter(ds["text"])))

In [ ]:
ds = ds.train_test_split(test_size=0.2, seed=42)
ds_train = ds["train"]
ds_eval = ds["test"]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")

In [ ]:
lora_config = LoraConfig(
    r = 16,
    lora_alpha = 32,
    lora_dropout = 0.05,
    bias = "none",
    task_type = "CAUSAL_LM",
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained("facebook/opt-1.3b", torch_dtype = torch.float16, device_map="auto")

In [ ]:
for param in model.parameters():
  param.requires_grad = False
  if param.ndim == 1:
    param.data = param.data.to(torch.float16)

model.gradient_checkpointing_enable
model.enable_input_require_grads

class OutputToFloat(nn.Sequential):
  def forward(self, x):
    return super().forward(x).to(torch.float16)

model.lm_head = OutputToFloat(model.lm_head)

In [ ]:
training_args = TrainingArguments(
    output_dir="./OPT-fine_tuned-FinGPT-CPU",
    dataloader_drop_last=True,
    max_steps = 10,
    save_strategy="steps",
    num_train_epochs=10,
    logging_steps=5,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    learning_rate=1e-4,
    lr_scheduler_type="cosine",
    warmup_steps=100,
    gradient_accumulation_steps=1,
    gradient_checkpointing=False,
    fp16=False,
    bf16=True,
    weight_decay=0.05,
    ddp_find_unused_parameters=False,
    run_name="OPT-fine_tuned-FinGPT-CPU",
    report_to="wandb",
)

In [ ]:
trainer = SFTTrainer(
    model = model,
    args = training_args,
    train_dataset = ds_train,
    eval_dataset = ds_eval,
    peft_config = lora_config,
)

print("Training...")
trainer.train()

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
  "facebook/opt-1.3b", return_dict=True, torch_dtype=torch.bfloat16
)

In [ ]:
model = PeftModel.from_pretrained(model, "./OPT-fine_tuned-FinGPT-CPU/checkpoint-10/")
model.eval()
model = model.merge_and_unload()
model.save_pretrained("./OPT-fine_tuned-FinGPT-CPU/merged")